## Similar buyers

### Collaborative filtering algorithm

A collaborative filtering algorithm usually works by searching a large group of people
and finding a smaller set with tastes similar to yours. It looks at other things they
like and combines them to create a ranked list of suggestions

#### 1. Collecting Preferences

(If we have a shopping site, we can use a value of 3 to indicate that someone had bought an item, 2 - item is in wish list, 1 - someone browsed item.)

In [1]:
shopping = {
    'Lisa Rose': {
        'notebook': 3,
        'e-book': 1,
        'phone': 1,
        'bag': 2,
        'mouse': 3,
        'shampoo': 1,
    },
    'Gene Seymour': {
        'notebook': 2,
        'e-book': 3,
        'phone': 1,
        'bag': 3,
        'mouse': 2,
        'sock': 2,
        'shoes': 2,
        'shirt': 2,
        'shampoo': 1,
    },
    'Michael Phillips': {
        'notebook': 2,
        'bag': 2,
        'mouse': 3,
        'shampoo': 1,
    },
    'Claudia Puig': {
        'notebook': 1,
        'bag': 3,
        'mouse': 2,
        'shampoo': 3,
    },
    'Mick LaSalle': {
        'table': 2,
        'sock': 1,
        'shoes': 3,
        'shirt': 1,
        'cap': 2,
        'shampoo': 1,
    },
    'Lary Broun': {
        'mouse': 1,
        'sock': 2,
        'shoes': 3,
        'shirt': 3,
        'cap': 1,
        'shampoo': 1,
     },
    'Jack Matthews': {
        'notebook': 2,
        'e-book': 3,
        'phone': 1,
        'bag': 2,
    },
    'Toby': {
        'mouse': 1,
        'sock': 3,
        'shoes': 3,
        'shirt': 3,
        'cap': 1,
        'shampoo': 1,
},
}


In [39]:
shopping['Lisa Rose']['notebook']

3

In [40]:
shopping['Toby']

{'cap': 1, 'mouse': 1, 'shampoo': 1, 'shirt': 3, 'shoes': 3, 'sock': 3}

#### 2. Finding Similar Customers

Euclidean Distance Score

In [9]:
from math import sqrt


In [2]:
# Returns a distance-based similarity score for person1 and person2
def sim_distance(prefs,person1,person2):
    
    # Get the list of shared_items
    si={}
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item]=1
            
    # if they have no purchase in common, return 0
    if len(si)==0: return 0
    
    # Add up the squares of all the differences
    sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2) for item in prefs[person1] if item in prefs[person2]])
    return 1.0/(1.0+float(sum_of_squares))

In [45]:
sim_distance(shopping, 'Mick LaSalle','Toby')

0.1

In [44]:
sim_distance(shopping, 'Gene Seymour','Toby')

0.2

In [54]:
sim_distance(shopping, 'Lary Broun','Toby')

0.5

#### 3. Pearson Correlation Score

In [14]:
# Returns the Pearson correlation coefficient for p1 and p2
def sim_pearson(prefs,p1,p2):
    # Get the list of mutually rated items
    si={}
    for item in prefs[p1]:
        if item in prefs[p2]:
            si[item]=1
            
    # Find the number of elements
    n=len(si)
    
    # if they are no purchase in common, return 0
    if n==0: return 0
    
    # Add up all the preferences
    sum1=sum([prefs[p1][it] for it in si])
    sum2=sum([prefs[p2][it] for it in si])
    
    # Sum up the squares
    sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
    sum2Sq=sum([pow(prefs[p2][it],2) for it in si])
    
    # Sum up the products
    pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
    
    # Calculate Pearson score
    num=pSum-(sum1*sum2/n)
    den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    
    if den==0: return 0
    r=num/den
    return r

In [46]:
sim_pearson(shopping, 'Mick LaSalle','Toby')

0.22360679774997896

In [47]:
sim_pearson(shopping, 'Gene Seymour','Toby')

0.8944271909999159

In [55]:
sim_pearson(shopping, 'Lary Broun','Toby')

0.9128709291752769

#### 4. Ranking the shopping.

In [69]:
# Returns the best matches for person from the prefs dictionary.
# Number of results and similarity function are optional params.
def topMatches(prefs,person,n=5,similarity=sim_pearson):
    scores=[(similarity(prefs,person,other),other)  for other in prefs if other!=person]
    
    # Sort the list so the highest scores appear at the top
    scores.sort( )
    scores.reverse( )
    return scores[0:n]

In [57]:
topMatches(shopping,'Toby',n=5)

[(0.9128709291752769, 'Lary Broun'),
 (0.8944271909999159, 'Gene Seymour'),
 (0.22360679774997896, 'Mick LaSalle'),
 (0, 'Michael Phillips'),
 (0, 'Lisa Rose')]

#### 5. Recommending Items

In [58]:
# Gets recommendations for a person by using a weighted average
# of every other user's rankings
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals={}
    simSums={}
    for other in prefs:
        
        # don't compare me to myself
        if other==person: 
            continue 
            
        sim=similarity(prefs,person,other)
        
        # ignore scores of zero or lower
        if sim<=0: 
            continue
            
        for item in prefs[other]:
            # only score movies I haven't seen yet
            
            if item not in prefs[person] or prefs[person][item]==0:
                
                # Similarity * Score
                totals.setdefault(item,0)
                totals[item]+=prefs[other][item]*sim
                
                # Sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+=sim
                
                
                
    # Create the normalized list
    rankings=[(total/simSums[item],item) for item,total in totals.items( )]

    # Return the sorted list
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [59]:
getRecommendations(shopping,'Toby')

[(3.0, 'e-book'),
 (3.0, 'bag'),
 (2.0, 'table'),
 (2.0, 'notebook'),
 (1.0, 'phone')]

In [60]:
getRecommendations(shopping,'Toby', similarity=sim_distance)

[(2.4782608695652173, 'bag'),
 (2.043478260869566, 'notebook'),
 (2.0, 'table'),
 (2.0, 'e-book'),
 (1.0, 'phone')]

## Matching Products

Here we will try to see which products are similar to each other. In this case we must transform the dataset to production == > people ==> number. It seems just to swap the people and the items.

#### 1. Similar products

In [61]:
def transformPrefs(prefs):
    result={}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
            
            # Flip item and person
            result[item][person]=prefs[person][item]
    return result

In [63]:
products = transformPrefs(shopping)
products

{'bag': {'Claudia Puig': 3,
  'Gene Seymour': 3,
  'Jack Matthews': 2,
  'Lisa Rose': 2,
  'Michael Phillips': 2},
 'cap': {'Lary Broun': 1, 'Mick LaSalle': 2, 'Toby': 1},
 'e-book': {'Gene Seymour': 3, 'Jack Matthews': 3, 'Lisa Rose': 1},
 'mouse': {'Claudia Puig': 2,
  'Gene Seymour': 2,
  'Lary Broun': 1,
  'Lisa Rose': 3,
  'Michael Phillips': 3,
  'Toby': 1},
 'notebook': {'Claudia Puig': 1,
  'Gene Seymour': 2,
  'Jack Matthews': 2,
  'Lisa Rose': 3,
  'Michael Phillips': 2},
 'phone': {'Gene Seymour': 1, 'Jack Matthews': 1, 'Lisa Rose': 1},
 'shampoo': {'Claudia Puig': 3,
  'Gene Seymour': 1,
  'Lary Broun': 1,
  'Lisa Rose': 1,
  'Michael Phillips': 1,
  'Mick LaSalle': 1,
  'Toby': 1},
 'shirt': {'Gene Seymour': 2, 'Lary Broun': 3, 'Mick LaSalle': 1, 'Toby': 3},
 'shoes': {'Gene Seymour': 2, 'Lary Broun': 3, 'Mick LaSalle': 3, 'Toby': 3},
 'sock': {'Gene Seymour': 2, 'Lary Broun': 2, 'Mick LaSalle': 1, 'Toby': 3},
 'table': {'Mick LaSalle': 2}}

find the set of products most similar to notebook

In [64]:
topMatches(products,'notebook')

[(0.7071067811865475, 'mouse'),
 (0, 'table'),
 (0, 'sock'),
 (0, 'shoes'),
 (0, 'shirt')]

In [65]:
topMatches(products,'sock')

[(0.8164965809277261, 'shirt'),
 (0, 'table'),
 (0.0, 'shoes'),
 (0, 'shampoo'),
 (0, 'phone')]

In [67]:
topMatches(products,'bag')

[(0.5773502691896258, 'shampoo'),
 (0.5773502691896258, 'e-book'),
 (0, 'table'),
 (0, 'sock'),
 (0, 'shoes')]

get list of people who buy this products in future

In [70]:
getRecommendations(products,'notebook')

[(1.0, 'Toby'), (1.0, 'Lary Broun')]

In [71]:
getRecommendations(products,'mouse')

[(2.0, 'Jack Matthews')]

#### 2. Similarity distance for products.

In [73]:
def calculateSimilarItems(prefs,n=10):
    
   # Create a dictionary of items showing which other items they
   # are most similar to.
   result={}
    
   # Invert the preference matrix to be item-centric
   itemPrefs=transformPrefs(prefs)
   c=0
   for item in itemPrefs:
        
       # Status updates for large datasets
       c+=1
       if c%100==0: print "%d / %d" % (c,len(itemPrefs))
            
       # Find the most similar items to this one
       scores=topMatches(itemPrefs,item,n=n,similarity=sim_distance)
       result[item]=scores
            
   return result

In [78]:
prod_sim = calculateSimilarItems(shopping)
prod_sim

{'bag': [(0.5, 'sock'),
  (0.5, 'shoes'),
  (0.5, 'shirt'),
  (0.3333333333333333, 'e-book'),
  (0.2, 'mouse'),
  (0.14285714285714285, 'shampoo'),
  (0.14285714285714285, 'phone'),
  (0.14285714285714285, 'notebook'),
  (0, 'table'),
  (0, 'cap')],
 'cap': [(1.0, 'table'),
  (1.0, 'mouse'),
  (0.5, 'shampoo'),
  (0.14285714285714285, 'sock'),
  (0.1, 'shoes'),
  (0.1, 'shirt'),
  (0, 'phone'),
  (0, 'notebook'),
  (0, 'e-book'),
  (0, 'bag')],
 'e-book': [(0.5, 'sock'),
  (0.5, 'shoes'),
  (0.5, 'shirt'),
  (0.3333333333333333, 'bag'),
  (0.2, 'shampoo'),
  (0.16666666666666666, 'mouse'),
  (0.14285714285714285, 'notebook'),
  (0.1111111111111111, 'phone'),
  (0, 'table'),
  (0, 'cap')],
 'mouse': [(1.0, 'cap'),
  (0.3333333333333333, 'notebook'),
  (0.2, 'bag'),
  (0.16666666666666666, 'sock'),
  (0.16666666666666666, 'phone'),
  (0.16666666666666666, 'e-book'),
  (0.1111111111111111, 'shoes'),
  (0.1111111111111111, 'shirt'),
  (0.09090909090909091, 'shampoo'),
  (0, 'table')],
 'no

#### 3. Getting Recommendations for products. 

In [75]:
def getRecommendedItems(prefs,itemMatch,user):
    userRatings=prefs[user]
    scores={}
    totalSim={}
    
    # Loop over products rated by this user
    for (item,rating) in userRatings.items( ):
        
        # Loop over items similar to this one        
        for (similarity,item2) in itemMatch[item]:
            
            # Ignore if this user has already rated this product            
            if item2 in userRatings: continue
                
            # Weighted sum of rating times similarity            
            scores.setdefault(item2,0)
            scores[item2]+=similarity*rating
            
            # Sum of all the similarities
            totalSim.setdefault(item2,0)
            totalSim[item2]+=similarity
            
    # Divide each total score by total weighting to get an average
    rankings=[(score/totalSim[item],item) for item,score in scores.items( )]
    
    # Return the rankings from highest to lowest
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [79]:
getRecommendedItems(shopping, prod_sim, 'Toby')

[(2.7522123893805306, 'notebook'),
 (2.6279069767441867, 'bag'),
 (2.607142857142857, 'e-book'),
 (2.1250000000000004, 'phone'),
 (2.0, 'table')]